## 1. Setup

Import the parser and related components.

In [ ]:
import sys
sys.path.insert(0, '/home/stellar/PublicRepo/YMHY/anime_streamer_2_0')

from cores.flow_core.tokenizer import tokenize
from cores.flow_core.parser import Parser, parse
from cores.flow_core.models import (
    FlowFile, FlowNode, NodeRef, FileRef, 
    ImportNode, Assignment, FlowParams, FlowStyle
)

# Create a parser instance
parser = Parser()

print("✓ Parser imported successfully")

## 2. Basic Parsing

A simple example showing how tokens become an AST.

In [ ]:
source = '''
@greeting |<<<Hello, World!>>>|.
@out |$greeting|.
'''

# Tokenize first
tokens = tokenize(source)

# Then parse
flow_file = parse(tokens)

print(f"Parsed FlowFile:")
print(f"  Imports: {len(flow_file.imports)}")
print(f"  Nodes: {list(flow_file.nodes.keys())}")
print(f"  Assignments: {len(flow_file.assignments)}")
print(f"  Has @out: {flow_file.out_node is not None}")

## 3. Visualizing the AST

Let's create a helper function to pretty-print the AST structure.

In [ ]:
def visualize_node(node: FlowNode, indent: int = 0) -> None:
    """Pretty-print a FlowNode and its structure."""
    prefix = "  " * indent
    print(f"{prefix}@{node.id} (layer={node.layer})")
    
    # Print params if any non-default
    if node.params.style or node.params.mutable or node.params.llm_strategy:
        print(f"{prefix}  Params:")
        if node.params.style:
            style = node.params.style
            if style.title:
                print(f"{prefix}    style.title = {style.title!r}")
            if style.divider:
                print(f"{prefix}    style.divider = {style.divider}")
            if style.list_type:
                print(f"{prefix}    style.list = {style.list_type!r}")
        if node.params.mutable:
            print(f"{prefix}    mutable = True")
    
    # Print content
    if node.content:
        print(f"{prefix}  Content ({len(node.content)} items):")
        for i, item in enumerate(node.content):
            if isinstance(item, str):
                display = item[:40] + "..." if len(item) > 40 else item
                print(f"{prefix}    [{i}] STR: {display!r}")
            elif isinstance(item, NodeRef):
                ref_symbol = "^" if item.is_forward else "$"
                print(f"{prefix}    [{i}] REF: {ref_symbol}{item.id}")
            elif isinstance(item, FileRef):
                print(f"{prefix}    [{i}] FILE: ++{item.path}")
            elif isinstance(item, FlowNode):
                print(f"{prefix}    [{i}] NESTED:")
                visualize_node(item, indent + 3)
    
    # Print slots
    if node.slots:
        print(f"{prefix}  Slots ({len(node.slots)}):")
        for slot_name, slot_node in node.slots.items():
            print(f"{prefix}    @{slot_name}:")
            visualize_node(slot_node, indent + 3)

def visualize_flow_file(flow_file: FlowFile) -> None:
    """Pretty-print an entire FlowFile AST."""
    print("=" * 60)
    print("FlowFile AST")
    print("=" * 60)
    
    if flow_file.imports:
        print(f"\nImports ({len(flow_file.imports)}):")
        for imp in flow_file.imports:
            selectors = f" [{', '.join(imp.selectors)}]" if imp.selectors else ""
            print(f"  +{imp.path}{selectors}")
    
    if flow_file.nodes:
        print(f"\nNodes ({len(flow_file.nodes)}):")
        for node_id, node in flow_file.nodes.items():
            visualize_node(node, indent=1)
    
    if flow_file.assignments:
        print(f"\nAssignments ({len(flow_file.assignments)}):")
        for assign in flow_file.assignments:
            print(f"  ${assign.target} = ${assign.source}")
    
    print("\n" + "=" * 60)

print("✓ Visualization helpers defined")

In [ ]:
# Let's visualize our simple example
visualize_flow_file(flow_file)

## 4. Complex AST Examples

Let's parse some more complex examples to see the full AST structure.

### 4.1 Node with Style Parameters

In [ ]:
source = '''
@header
|style.title=<<Welcome>>
|style.divider=<<true>>
|<<<Main content goes here>>>
|.

@out |$header|.
'''

tokens = tokenize(source)
flow_file = parse(tokens)
visualize_flow_file(flow_file)

### 4.2 Nodes with References

In [ ]:
source = '''
@first |<<<First node>>>|.
@second |<<<References: >>>|$first|<<<and forward: >>>|^third|.
@third |<<<Third node>>>|.
@out |$second|.
'''

tokens = tokenize(source)
flow_file = parse(tokens)
visualize_flow_file(flow_file)

### 4.3 Nested Nodes with Slots

In [ ]:
source = '''
@document
|@header |<<<Title>>>|.
|@body |<<<Main content>>>|.
|@footer |<<<Footer>>>|.
|.

@out |$document|.
'''

tokens = tokenize(source)
flow_file = parse(tokens)
visualize_flow_file(flow_file)

### 4.4 File References and Imports

In [ ]:
source = '''
+./utils.flow |$helper|.
+./common.flow |.

@main |<<<Content: >>>|++./README.md|.
@out |$main|.
'''

tokens = tokenize(source)
flow_file = parse(tokens)
visualize_flow_file(flow_file)

### 4.5 Assignments

In [ ]:
source = '''
@template
|@placeholder |<<<Default content>>>|.
|.

@custom_content |<<<Custom value>>>|.

$template.placeholder = $custom_content

@out |$template|.
'''

tokens = tokenize(source)
flow_file = parse(tokens)
visualize_flow_file(flow_file)

## 5. Parse Sample Files

Let's parse the sample `.flow` files from the samples directory.

In [ ]:
from pathlib import Path

samples_dir = Path('./samples')
if samples_dir.exists():
    for sample_file in sorted(samples_dir.glob('*.flow')):
        print(f"\n{'#' * 60}")
        print(f"# {sample_file.name}")
        print(f"{'#' * 60}")
        
        source = sample_file.read_text()
        try:
            tokens = tokenize(source)
            flow_file = parse(tokens)
            print(f"\nNodes: {list(flow_file.nodes.keys())}")
            print(f"Imports: {len(flow_file.imports)}")
            print(f"Assignments: {len(flow_file.assignments)}")
            print(f"Has @out: {flow_file.out_node is not None}")
        except Exception as e:
            print(f"Error parsing: {e}")
else:
    print("Samples directory not found")

## 6. Interactive Parser

Use this cell to experiment with your own Flow source code.

In [ ]:
# Your FLOW source here
source = '''
@greeting |<<<Hello!>>>|.
@out |$greeting|.
'''

try:
    tokens = tokenize(source)
    flow_file = parse(tokens)
    visualize_flow_file(flow_file)
except Exception as e:
    print(f"Error: {e}")